# Exercise 1

### In this first exercise, we will learn the **basics** of polars: let's start by loading a simple `DataFrame`.

### All examples below come from the polars `Getting Started` section.

## Reading and Writing

In [52]:
import polars as pl
from datetime import date, timedelta, datetime

df = pl.DataFrame(
    {
        "integer": [1, 2, 3],
        "date": [
            date(2025, 1, 1),
            date(2025, 1, 2),
            date(2025
            , 1, 3),
        ],
        "float": [4.0, 5.0, 6.0],
        "string": ["a", "b", "c"],
    }
)

print(df)

shape: (3, 4)
┌─────────┬────────────┬───────┬────────┐
│ integer ┆ date       ┆ float ┆ string │
│ ---     ┆ ---        ┆ ---   ┆ ---    │
│ i64     ┆ date       ┆ f64   ┆ str    │
╞═════════╪════════════╪═══════╪════════╡
│ 1       ┆ 2025-01-01 ┆ 4.0   ┆ a      │
│ 2       ┆ 2025-01-02 ┆ 5.0   ┆ b      │
│ 3       ┆ 2025-01-03 ┆ 6.0   ┆ c      │
└─────────┴────────────┴───────┴────────┘


### As you can see, `DataFrame` creation is quite similar to pandas.
### Also, you can see that polars natively supports most python types and prints them with the table.
### Now let's write this `DataFrame` to disk

In [23]:
df.write_csv("output.csv")
df = pl.read_csv("output.csv")
print(df)

shape: (3, 4)
┌─────────┬────────────┬───────┬────────┐
│ integer ┆ date       ┆ float ┆ string │
│ ---     ┆ ---        ┆ ---   ┆ ---    │
│ i64     ┆ str        ┆ f64   ┆ str    │
╞═════════╪════════════╪═══════╪════════╡
│ 1       ┆ 2025-01-01 ┆ 4.0   ┆ a      │
│ 2       ┆ 2025-01-02 ┆ 5.0   ┆ b      │
│ 3       ┆ 2025-01-03 ┆ 6.0   ┆ c      │
└─────────┴────────────┴───────┴────────┘


### As you can see, the date type format was lost when writing and re-opening the dataframe. Let's get it back in the next section: `Expressions`

## `Expressions`

`Expressions` are the core strength of Polars. The `expressions` offer a modular structure that allows you to combine simple concepts into complex queries. Below we cover the basic components that serve as building blocks (or in Polars terminology contexts) for all your queries:

- `select` => select certain columns (can include a condition on name)
- `filter` => filter rows based one or several conditions over the columns
- `with_columns` => add new columns to the dataframe based on existing ones.
- `group_by` => group rows by certain columns and aggregate them using any aggregation function (e.g. `sum`, `mean`) or used-defined aggregation (using `.agg`)

`Note`: in polars there are no such things as a row index (like in pandas), but it's still possible to select certain rows by their index (e.g. `df[0]` or `df[a:b]`)

## `with_columns`

### Let's compute the date as datetime format using `with_columns` (creates a new column) and `Expressions` (e.g. `pl.col`)

In [24]:
df = df.with_columns(
    pl.col('date').str.strptime(pl.Date, format='%Y-%m-%d')
)
print(df)

shape: (3, 4)
┌─────────┬────────────┬───────┬────────┐
│ integer ┆ date       ┆ float ┆ string │
│ ---     ┆ ---        ┆ ---   ┆ ---    │
│ i64     ┆ date       ┆ f64   ┆ str    │
╞═════════╪════════════╪═══════╪════════╡
│ 1       ┆ 2025-01-01 ┆ 4.0   ┆ a      │
│ 2       ┆ 2025-01-02 ┆ 5.0   ┆ b      │
│ 3       ┆ 2025-01-03 ┆ 6.0   ┆ c      │
└─────────┴────────────┴───────┴────────┘


### as you can see, the `date` column now is of type `date` instead of `str`, which means that we can apply transformations specific to the `dt` `namespace`.

### In polars, we call a `namespace` any collection of methods that operate on a certain data type. For instance, we just used the `str` namespace in order to transform our `str` type date column into a `date` type date column using the `strptime` method that only applies to `str` data.

### Now let's use the `dt` namespace (temporal data) and `with_columns` to add 5 business days to our `date` column.

### For this one we will create a new column by chaining the `.name.suffix` method.

In [53]:
df = df.with_columns(
    pl.col('date').dt.add_business_days(n=1, roll='forward').name.suffix('_next_business_day')
)
print(df)

shape: (3, 5)
┌─────────┬────────────┬───────┬────────┬────────────────────────┐
│ integer ┆ date       ┆ float ┆ string ┆ date_next_business_day │
│ ---     ┆ ---        ┆ ---   ┆ ---    ┆ ---                    │
│ i64     ┆ date       ┆ f64   ┆ str    ┆ date                   │
╞═════════╪════════════╪═══════╪════════╪════════════════════════╡
│ 1       ┆ 2025-01-01 ┆ 4.0   ┆ a      ┆ 2025-01-02             │
│ 2       ┆ 2025-01-02 ┆ 5.0   ┆ b      ┆ 2025-01-03             │
│ 3       ┆ 2025-01-03 ┆ 6.0   ┆ c      ┆ 2025-01-06             │
└─────────┴────────────┴───────┴────────┴────────────────────────┘


### Another way to do that is to include the new column's name at the beginning

In [29]:
df = df.with_columns(
    date_next_business_day=pl.col('date').dt.add_business_days(n=1, roll='forward')
)
print(df)

shape: (3, 5)
┌─────────┬────────────┬───────┬────────┬────────────────────────┐
│ integer ┆ date       ┆ float ┆ string ┆ date_next_business_day │
│ ---     ┆ ---        ┆ ---   ┆ ---    ┆ ---                    │
│ i64     ┆ date       ┆ f64   ┆ str    ┆ date                   │
╞═════════╪════════════╪═══════╪════════╪════════════════════════╡
│ 1       ┆ 2025-01-01 ┆ 4.0   ┆ a      ┆ 2025-01-02             │
│ 2       ┆ 2025-01-02 ┆ 5.0   ┆ b      ┆ 2025-01-03             │
│ 3       ┆ 2025-01-03 ┆ 6.0   ┆ c      ┆ 2025-01-06             │
└─────────┴────────────┴───────┴────────┴────────────────────────┘


## `select`

### `select` can be used to select columns based on name conditions. Let's start by selecting all columns with `pl.col(*)`

In [27]:
df.select(
    pl.col('*')
)

integer,date,float,string,date_next_business_day
i64,date,f64,str,date
1,2025-01-01,4.0,"""a""",2025-01-02
2,2025-01-02,5.0,"""b""",2025-01-03
3,2025-01-03,6.0,"""c""",2025-01-06


### Now let's select several columns

In [28]:
df.select(
    pl.col('float', 'string')
)

float,string
f64,str
4.0,"""a"""
5.0,"""b"""
6.0,"""c"""


### We can also select columns with a regex on name
### In polars, regexes start with `^`, end with `$` and `*` are preceded by `.`.

In [44]:
df.select(
    pl.col('^.*date.*$')
)

date,date_next_business_day
date,date
2025-01-01,2025-01-02
2025-01-02,2025-01-03
2025-01-03,2025-01-06


## `filter`

### Filters rows based on a condition.

In [49]:
df.filter(
    pl.col('string').str.contains('c'),
)

integer,date,float,string,date_next_business_day
i64,date,f64,str,date
3,2025-01-03,6.0,"""c""",2025-01-06


### Having multiple expressions inside a filter works as an `and` statement.

In [56]:
df.filter(
    pl.col('date').dt.offset_by('1d').eq(pl.col('date_next_business_day')),
    pl.col('float') > 4.0,
)

integer,date,float,string,date_next_business_day
i64,date,f64,str,date
2,2025-01-02,5.0,"""b""",2025-01-03


## `group_by` and `agg`
### as in pandas, `group_by` (notice the `_`) and `agg` allow to group based on variable value and perform an operation on said groups.

In [59]:
# NOTE: polars allow you to choose unsigned int (positive integers, and precision according to your needs)

df = df.with_columns(pl.Series('id', dtype=pl.UInt8, values=[1,1,2]))
print(df)

shape: (3, 6)
┌─────────┬────────────┬───────┬────────┬────────────────────────┬─────┐
│ integer ┆ date       ┆ float ┆ string ┆ date_next_business_day ┆ id  │
│ ---     ┆ ---        ┆ ---   ┆ ---    ┆ ---                    ┆ --- │
│ i64     ┆ date       ┆ f64   ┆ str    ┆ date                   ┆ u8  │
╞═════════╪════════════╪═══════╪════════╪════════════════════════╪═════╡
│ 1       ┆ 2025-01-01 ┆ 4.0   ┆ a      ┆ 2025-01-02             ┆ 1   │
│ 2       ┆ 2025-01-02 ┆ 5.0   ┆ b      ┆ 2025-01-03             ┆ 1   │
│ 3       ┆ 2025-01-03 ┆ 6.0   ┆ c      ┆ 2025-01-06             ┆ 2   │
└─────────┴────────────┴───────┴────────┴────────────────────────┴─────┘


In [66]:
df.group_by('id').agg(
    pl.col('string').str.concat(delimiter='').name.suffix('s_concatenated'),
    pl.col('float').mean().name.suffix('_mean'),
    pl.col('integer').last().name.prefix('last_'),
    pl.col('date').min().name.prefix('first_'),
)

id,strings_concatenated,float_mean,last_integer,first_date
u8,str,f64,i64,date
2,"""c""",6.0,3,2025-01-03
1,"""ab""",4.5,2,2025-01-01


### Now let's move on to some **real** exercises!